# Bindings

There are two good reason for leaving the Python universe and trying to reach into another one: Either there is already stable library that solves the problem significantly faster or there is good reason to believe that rewriting parts of a Python program in a compiled language yields performance benefits. The hurdles that have to be overcome in order to do this are __type marshaling__, __calling conventions__ and __memory management__. Concepts from one language often do not easily translate into those of other languages, therefore, depending on the languages that need to be glued together, the translation effort more or less severe. Fortunately, there are tools available that carry out the dirty work for the programmer.

## C Foreign Function Interface for Python (CFFI)

[CFFI](https://cffi.readthedocs.io/en/latest/index.html) is primarliy used as an interface to pre-compiled shared objects, e.g. a domain-specific C-library that needs an easy-to-use Python wrapper. It does not require to learn a new wrapper language.
The main modes in `cffi` are the API (application programming interface) and the ABI mode. Besides that there are two sub-modes: in-line and out-of-line mode.
In the __API  mode__, a C wrapper for CPython is compiled and the targeted funtions are invoked directly. This is significantly faster than the __ABI mode__ and also makes use of the compiler's support. On the other hand, the ABI mode requries function calls to go through [libffi](https://sourceware.org/libffi/) which implies significant overhead. In the __in-line mode__, everything is set up each time the Python code is imported. In the __out-of-line__ mode, there are separate steps of preparation and compilation that produce a module which the main program can then import.

* `ffi.cdef` parses the given C source and registers all the functions, types, constants and global variables in the C source
* `ffi.cast` returns an instance of the C type initialized with the given value
* `ffi.from_buffer` meant to be used on objects containing large quantities of raw data, e.g. numpy arrays
* `ffi.set_source` prepares for producing out-of-line an external module
* `ffi.dlopen` opens a shared library

#### ABI in-line

In [1]:
from cffi import FFI


ffibuilder = FFI()
ffibuilder.cdef("""
    double exp(double x);
    double sqrt(double x);
""")
lib = ffibuilder.dlopen("libm.so.6")


expi = lib.exp(3.141592)
sqrtwo = lib.sqrt(2.0)

print(expi)
print(sqrtwo)

23.140677508263703
1.4142135623730951


#### API out-of-line

In [2]:
%%writefile bindings/cffi/dot_product_sqrt.c

# include <math.h>


double dot_product_sqrt(double *v, double *w, int n) {
  double p = 0.0;
    
  for (int i = 0; i < n; i++) {
      p += v[i]*w[i];
  }
    
  return sqrt(p);
}

Writing bindings/cffi/dot_product_sqrt.c


In [3]:
%%writefile bindings/cffi/dot_product_sqrt.h

double dot_product_sqrt(double *v, double *w, int n);

Writing bindings/cffi/dot_product_sqrt.h


In [4]:
%%writefile bindings/cffi/dot_product_sqrt.py

from cffi import FFI
ffibuilder = FFI()

ffibuilder.cdef("double dot_product_sqrt(double *v, double *w, int n);")

ffibuilder.set_source("_dot_product_sqrt",
"""
    #include "dot_product_sqrt.h"
""",
    sources=['dot_product_sqrt.c'],
    libraries=['m']) 

if __name__ == "__main__":
    ffibuilder.compile(verbose=True)

Writing bindings/cffi/dot_product_sqrt.py


In [5]:
! cd bindings/cffi; python3 dot_product_sqrt.py

generating ./_dot_product_sqrt.c
the current directory is '/home/fs70824/trainee19/python4hpc/bindings/cffi'
running build_ext
building '_dot_product_sqrt' extension
/opt/sw/spack-0.12.1/opt/spack/linux-centos7-x86_64/gcc-9.1.0/gcc-10.2.0-2aa5hfe7om4udww5qqzk4rdgzggt4w2t/bin/gcc -Wno-unused-result -Wsign-compare -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /opt/sw/vsc4/VSC/x86_64/generic/jupyterhub-envs/conda/envs/jupyterhub-dask/include -fPIC -O2 -isystem /opt/sw/vsc4/VSC/x86_64/generic/jupyterhub-envs/conda/envs/jupyterhub-dask/include -fPIC -I/opt/sw/vsc4/VSC/x86_64/generic/jupyterhub-envs/conda/envs/jupyterhub-dask/include/python3.10 -c _dot_product_sqrt.c -o ./_dot_product_sqrt.o
/opt/sw/spack-0.12.1/opt/spack/linux-centos7-x86_64/gcc-9.1.0/gcc-10.2.0-2aa5hfe7om4udww5qqzk4rdgzggt4w2t/bin/gcc -Wno-unused-result -Wsign-compare -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /opt/sw/vsc4/VSC/x86_64/generic/jupyterhub-envs/conda/envs/jupyterhub-dask/include -fPIC -O2 -isystem /opt/sw/v

In [6]:
%%writefile bindings/cffi/main.py

import numpy as np
from _dot_product_sqrt import ffi, lib


n = 10
v = np.ones(n)
w = np.ones_like(v)

vptr = ffi.cast("double *", ffi.from_buffer(v))
wptr = ffi.cast("double *", ffi.from_buffer(w))
res = lib.dot_product_sqrt(vptr, wptr, n)

print("sqrt(a*b) = ", res)

Writing bindings/cffi/main.py


In [7]:
! cd bindings/cffi; python3 main.py

sqrt(a*b) =  3.1622776601683795


## Cython

Cython, as described in another chapter, is both a programming language and a transpiler that translates Python Code into either C or C++. Conveniently, this gives it also the capability to wrap external C libraries with cythonic wrappers that essentially look like Cython programs. With relatively little effort, Cython wrappers deliver Python-friendly interfaces with minimal wrapper overhead that call external C libraries and make use of their performance. 


* `cdef` Cython functions that are intended to be pure C functions, all types must be declared
* `cimport` Used to import C data types, C functions, variables, and extension types


__References__
* Cython: A Guide for Python Programmers (2015) by Kurt Smith
* [Cython - Using C libraries](https://cython.readthedocs.io/en/stable/src/tutorial/clibraries.html)

In [8]:
%load_ext cython

In [9]:
%%writefile bindings/cython/dot_product_sqrt.c

# include <math.h>


double dot_product_sqrt(double *v, double *w, int n) {
  double p = 0.0;
    
  for (int i = 0; i < n; i++) {
      p += v[i]*w[i];
  }
    
  return sqrt(p);
}

Writing bindings/cython/dot_product_sqrt.c


In [10]:
%%writefile bindings/cython/dot_product_sqrt.h

double dot_product_sqrt(double *v, double *w, int n);

Writing bindings/cython/dot_product_sqrt.h


In [11]:
%%writefile bindings/cython/c_dot_product_sqrt.pxd

# cdefine of C function signature
cdef extern from "dot_product_sqrt.h":
    double dot_product_sqrt (double *v, double *w, int n)

Writing bindings/cython/c_dot_product_sqrt.pxd


In [12]:
%%writefile bindings/cython/py_dot_product_sqrt.pyx

from c_dot_product_sqrt cimport dot_product_sqrt # import declarations from *.pxd
cimport numpy as np # Cython declarations for numpy
np.import_array() # Cython Numpy-C-API


# wrapper code with numpy type annotations
def dot_product_sqrt_py(np.ndarray[double, ndim=1, mode="c"] v not None,
                        np.ndarray[double, ndim=1, mode="c"] w not None):
    return dot_product_sqrt(&v[0], &w[0], v.shape[0])

Writing bindings/cython/py_dot_product_sqrt.pyx


In [13]:
%%writefile bindings/cython/setup.py

import numpy as np
from distutils.core import setup
from distutils.extension import Extension
from Cython.Build import cythonize


sourcefiles = ["py_dot_product_sqrt.pyx", "dot_product_sqrt.c"]
extensions = [Extension("dot_product_sqrt", sourcefiles, include_dirs=[np.get_include()])]

setup(
    ext_modules=cythonize(extensions),
    include_dirs=[np.get_include()]
)

Writing bindings/cython/setup.py


In [14]:
! cd bindings/cython ; python3 setup.py build_ext --inplace

Compiling py_dot_product_sqrt.pyx because it changed.
[1/1] Cythonizing py_dot_product_sqrt.pyx
/opt/sw/vsc4/VSC/x86_64/generic/jupyterhub-envs/conda/envs/jupyterhub-dask/lib/python3.10/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/fs70824/trainee19/python4hpc/bindings/cython/py_dot_product_sqrt.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running build_ext
building 'dot_product_sqrt' extension
creating build
creating build/temp.linux-x86_64-cpython-310
/opt/sw/spack-0.12.1/opt/spack/linux-centos7-x86_64/gcc-9.1.0/gcc-10.2.0-2aa5hfe7om4udww5qqzk4rdgzggt4w2t/bin/gcc -Wno-unused-result -Wsign-compare -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /opt/sw/vsc4/VSC/x86_64/generic/jupyterhub-envs/conda/envs/jupyterhub-dask/include -fPIC -O2 -isystem /opt/sw/vsc4/VSC/x86_64/generic/jupyterhub-envs/conda/envs/jupyterhub-dask/include -fPIC -I. -I/opt/sw/vsc4/

In [15]:
%%writefile bindings/cython/main.py

import numpy as np
from dot_product_sqrt import dot_product_sqrt_py


n = 10
v = np.ones(n)
w = np.ones_like(v)

res = dot_product_sqrt_py(v, w)

print("sqrt(a*b) = ", res)

Writing bindings/cython/main.py


In [16]:
! cd bindings/cython ; python3 main.py

sqrt(a*b) =  3.1622776601683795


## f2py - Fortran to Python interface generator

The gold standard for scientific applications in many domains is still (modern) Fortran; therefore binding Python to Fortran or developing a Python wrapper for Fortran modules might come in handy. [f2py](https://numpy.org/doc/stable/f2py/), which is part of NumPy, generates a Python C/API files that contain wrappers forgiven fortran functions so that they can be called from Python.

Specifically, the following Fortran functions and structures are supported:
* Fortran 77/90/95 external subroutines
* Fortran 90/95 module subroutines
* C functions
* Fortran 77 COMMON blocks
* Fortran 90/95 module data
* allocatable arrays

However, Modern Fortran, i.e. Fortran 2003 and newer, supports interoperability with C in a standardized way using the `iso_c_binding` module. This opens up the possibility to use Fortran from Python through a C interface provided by CFFI or Cython. Naturally, this requires extra C code and there is no automatic tool yet.


___References___
* [Interfacing Modern Fortran with Python](https://www.fortran90.org/src/best-practices.html#interfacing-with-python)
* [f2py Reference Manual](https://numpy.org/doc/stable/f2py/index.html)

In [17]:
%%writefile bindings/f2py/fibonacci.f90

subroutine fib(a,n)
    integer, intent(in) :: n
    integer :: i
    real(8) :: a(n)
    do i=1, n
        if (i==1) then
            a(i) = 0.0d0
        else if (i==2) then
            a(i) = 1.0d0
        else
            a(i) = a(i-1) + a(i-2)
        end if
    end do
end subroutine

Writing bindings/f2py/fibonacci.f90


In [18]:
! cd bindings/f2py ; f2py3 --verbose --build-dir . -c fibonacci.f90 -m fibm

No `name` configuration, performing automatic discovery
running build
running config_cc
INFO: unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
INFO: unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
INFO: build_src
INFO: building extension "fibm" sources
INFO: f2py options: []
INFO: f2py:> ./src.linux-x86_64-3.10/fibmmodule.c
creating src.linux-x86_64-3.10
Reading fortran codes...
	Reading file 'fibonacci.f90' (format:free)
Post-processing...
	Block: fibm
			Block: fib
Post-processing (stage 2)...
Building modules...
    Building module "fibm"...
        Constructing wrapper function "fib"...
          fib(a,[n])
    Wrote C/API module "fibm" to file "./src.linux-x86_64-3.10/fibmmodule.c"
INFO:   adding './src.linux-x86_64-3.10/./src.linux-x86_64-3.10/fortranobject.c' to sources.
INFO:   adding './src.linux-x86_64-3.10/./src.linux-x86_64-3.10' to include_dirs.
creating src.linux-x8

In [19]:
%%writefile bindings/f2py/fibpy.py

import fibm
import numpy as np


v = np.empty(10)
fibm.fib(v)

print(v)

Writing bindings/f2py/fibpy.py


In [20]:
! cd bindings/f2py ; python3 fibpy.py

[ 0.  1.  1.  2.  3.  5.  8. 13. 21. 34.]
